In [1]:
print("hello David")

hello David


In [ ]:
#!uv pip install yfinance --upgrade --no-cache-dir

In [3]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
import yfinance as yf

class YFinanceStockAnalysisInput(BaseModel):
    """Input schema for MyCustomTool."""
    ticker: str = Field(..., description="stock ticker symbol.")

class YFinanceStockAnalysisTool(BaseTool):
    name: str = "YFinance Stock Analysis Tool"
    description: str = (
        "fetch and analyse stocks for a given ticker using yfinance."
    )
    args_schema: Type[BaseModel] = YFinanceStockAnalysisInput

    def _run(self, ticker: str) -> str:
        stock = yf.Ticker(ticker)
        info = stock.info
        history = stock.history(period='5y')
        week_52_high = history['High'].tail(260).max()
        week_52_low = history['Low'].tail(260).min()
        financials = stock.financials
        
        if not financials.empty and 'Total Revenue' in financials.index:
            revenue_5y = financials.loc['Total Revenue'].iloc[:5]
            if len(revenue_5y) >= 5:  
                # ATTENTION : formule pas parfaite, on y reviendra
                revenue_growth = revenue_5y.iloc[0] / revenue_5y.iloc[-1] ** (1/5)
            else:
                revenue_growth = None
        else:
            revenue_growth = None
    
        analysis = {
            "ticker symbol": ticker,
            "company name": info.get('longName', "N/A"),
            "current price": info.get('currentPrice', 'N/A'),
            "52-week  high": round(week_52_high, 2),
            "52-week low": round(week_52_low, 2),
            "market cap": info.get('marketCap', 'N/A'),
            "P/E ratio": info.get('trailingPE', 'N/A'),
            "P/B ratio": info.get('priceToBook', 'N/A'),
            "Debt-to-equity ratio": info.get('debtToEquity', 'N/A'),
            "Current ratio": info.get('currentRatio', 'N/A'),
            "Dividend yield (%)": info.get('dividendYield', 'N/A'),
            "5-year revenue growth rate (%)": revenue_growth,
            "Free cash flow": info.get('freeCashflow', 'N/A'),
            "Profit margin": info.get('profitMargins', 'N/A'),
            "Operating margin": info.get('operatingMargins', 'N/A'),
            "Earning growth": info.get('earningsGrowth', 'N/A'),
            "Revenue growth": info.get('revenueGrowth', 'N/A'),
            "Analyst target price": info.get('targetMedianPrice', 'N/A'),
            "Beta": info.get('beta', 'N/A'),
            "5 year average return on equity (%)": info.get('returnOnEquity', 'N/A')
        }

        for key in [
            'Dividend yield (%)',
            '5-year revenue growth rate (%)',
            'Profit margin',
            'Operating margin',
            'Earning growth',
            'Revenue growth',
            '5 year average return on equity (%)'
        ]:
            if analysis[key] not in ['N/A', None]:
                analysis[key] = round(analysis[key] * 100, 2)
        
        output = "\n".join([f"{key}: {value}" for key, value in analysis.items()])

        return output

In [4]:
tool = YFinanceStockAnalysisTool()
print(tool._run("AAPL"))

ticker symbol: AAPL
company name: Apple Inc.
current price: 252.31
52-week  high: 259.18
52-week low: 168.8
market cap: 3744378650624
P/E ratio: 38.286797
P/B ratio: 56.941998
Debt-to-equity ratio: 154.486
Current ratio: 0.868
Dividend yield (%): 41.0
5-year revenue growth rate (%): nan
Free cash flow: 94873747456
Profit margin: 24.3
Operating margin: 29.99
Earning growth: 12.1
Revenue growth: 9.6
Analyst target price: 243.0
Beta: 1.109
5 year average return on equity (%): 149.81


In [6]:
import pandas as pd

output = tool._run("TSLA")
lines = [line.split(": ", 1) for line in output.splitlines()]
df = pd.DataFrame(lines, columns=["Metric", "Value"])
df

,Metric,Value
0,ticker symbol,TSLA
1,company name,"Tesla, Inc."
2,current price,425.85
3,52-week high,488.54
4,52-week low,212.11
5,market cap,1416015511552
6,P/E ratio,256.53616
7,P/B ratio,17.757809
8,Debt-to-equity ratio,16.823
9,Current ratio,2.037


In [10]:
import json

with open("..//outputs//compliance_report.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Ticker: ",data["ticker"])
print("Compliant: ",data["compliant"])
print("Issues: ",data["issues"])

Ticker:  ETH-USD
Compliant:  False
Issues:  Cryptocurrency or digital asset trading requires explicit authorization, which has not been provided for this order.


<img src="../docs/screenshots/agents_workflow.png" alt="Multi Agents Overview" width="800">